In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib widget

from pymeasure.instruments.srs import SR830

lockin_pym = SR830('GPIB2::25::INSTR')
lockin_pym.sample_frequency = 256
lockin_pym.write('FAST0')

# SR830 Code samples

## pyvisa

In [ ]:
import numpy as np
from pyvisa import ResourceManager
from pyvisa.resources import MessageBasedResource
from time import sleep
class SR830Interface:
    def __init__(self, address):
        # Implement your instrument initialization logic here
        # Example: open a connection to the instrument
        self.rm = ResourceManager()
        self.instrument = self.rm.open_resource(address)

    def buffer_stats(self, time):
        xbuf, ybuf = self.buffer_measure(time =time)
        return [np.mean(xbuf), np.std(xbuf)], [np.mean(ybuf), np.std(ybuf)]

    def buffer_measure(self, time):
        self.reset_buffer()
        self.start_buffer()
        sleep(time)
        self.pause_buffer()
        x_bytes, y_bytes =self.read_buffer_bytes()
        x, y = self.buffer_bytes_convert(x_bytes), self.buffer_bytes_convert(y_bytes)
        return x, y


    def buffer_bytes_convert(self, buffer):
        byteproduct = np.array(list(buffer[0::4])) + np.array(list(buffer[1::4]))*2**8
        divsor, remainder = np.divmod(
            byteproduct, 
            32768*np.ones(shape = byteproduct.shape)
        )
        mant = remainder - divsor*2**15
        exp = np.array(list(buffer[2::4]))
        return mant*np.power(np.ones(shape = exp.shape)*2, exp-124)

    def read_buffer_bytes(self):
        buffer_size = self.n_buff()
        self.instrument.write(f'TRCL?1, 0, {buffer_size}')
        x_bytes = self.instrument.read_raw()
        self.instrument.write(f'TRCL?2, 0, {buffer_size}')
        y_bytes = self.instrument.read_raw()

        return [x_bytes, y_bytes]
    
    def read_buffer_ascii(self):
        buffer_size = self.n_buff()
        x_buffer = self.instrument.query_ascii_values(f"TRCA? 1, 0, {buffer_size}")
        y_buffer = self.instrument.query_ascii_values(f"TRCA? 2, 0, {buffer_size}")
        return[x_buffer, y_buffer]

    def start_buffer(self):
        self.instrument.write("STRT")

    def pause_buffer(self):
        self.instrument.write("PAUS")

    def reset_buffer(self):
        self.instrument.write("REST")

    def snap(self):
        return self.instrument.query('SNAP? 1,2')
    
    def n_buff(self):
        return float(self.instrument.query('SPTS?'))
lockin = SR830Interface('GPIB2::25::INSTR')
xbuf, ybuf =lockin.buffer_measure(5)
fig, ax = plt.subplots(2,1)

sns.histplot(xbuf*1E3, ax = ax[0])
sns.histplot(ybuf*1E3, ax= ax[1])


## Pymeasure

In [ ]:
from pymeasure.instruments.srs import SR830

lockin = SR830('GPIB2::25::INSTR')
lockin.binary_values(f'TRCL?1,0, {lockin.buffer_count}')
lockin.values(f'TRCA?1,0, {lockin.buffer_count}', )
lockin.binary_values(f'TRCL?1,0, {lockin.buffer_count}')
xbuffer, ybuffer = lockin.get_buffer_binary(1), lockin.get_buffer_binary(2)
print(xbuffer.mean(), xbuffer.std())
print(ybuffer.mean(), ybuffer.std())
lockin.binary_values(f'TRCB?1,0,{lockin.buffer_count}')

lockin.write('FAST1')
xbuffer = np.array(lockin.get_buffer_float(channel = 1))
ybuffer = np.array(lockin.get_buffer_float(channel = 2))
print(xbuffer.mean(), xbuffer.std())
print(ybuffer.mean(), ybuffer.std())
print(lockin.SAMPLE_FREQUENCIES)
lockin.time_constant = 0.1
lockin.id
lockin.sample_frequency = 16
print(f'lockin sample freq is={lockin.sample_frequency}')
lockin.reset_buffer()
lockin.start_buffer()
lockin.wait_for_buffer(count = 10*lockin.sample_frequency)
print(lockin.buffer_count)
# xbuffer =lockin.get_buffer_binary(1)
# ybuffer= lockin.get_buffer_binary(2)
# lockin.write(f'TRCB?1,0,{lockin.buffer_count}')
# # lockin.read_binary_values()
# bytes =lockin.read_bytes(count =-1)
# np.frombuffer(bytes[0:], dtype = np.float32)

xbuffer = lockin.get_buffer_float(channel = 1)
ybuffer = lockin.get_buffer_float(channel = 2)
fig, [axa, axb] = plt.subplots(2, 1, tight_layout=True)
axa.hist(xbuffer, edgecolor = 'k')
axb.hist(ybuffer, edgecolor = 'k')
# lockin.write(f'TRCA?1,0,{lockin.buffer_count}')
# # time.sleep(5)
# xbuffer_str = lockin.read()
# xbuffer_str = lockin.ask(f'TRCA?1,0,{lockin.buffer_count}',)
# xbuffer_str

# lockin.write(f'TRCB?1,0,{lockin.buffer_count}')
# # time.sleep(5)
# xbuffer_str = lockin.read_bytes(-1)
# lockin.sample_frequency = 512
# print(f'lockin sample freq is={lockin.sample_frequency}')
# lockin.reset_buffer()
# lockin.start_buffer()
# lockin.wait_for_buffer(count = lockin.sample_frequency*10)
# print(lockin.buffer_count)
# lockin.write(f'TRCA?1,0,{lockin.buffer_count}')
# lockin.read()